<h1>Applied Data Science Capstone Final Assignment:</h1>

<h2>map.py</h2>

In [23]:
#!pip install folium
import json
import foursquare
import pandas as pd
from branca.element import CssLink, Figure, JavascriptLink, MacroElement
from branca.element import IFrame
from folium import GeoJson, Map, FeatureGroup, Popup, Marker
from folium.utilities import parse_options
from geopy import Nominatim
from jinja2 import Template
from geopy.exc import GeocoderTimedOut
import geopy

def do_geocode(add):
    try:
        return geopy.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

#provides the 'find my location' functionality.
class LocateControl(MacroElement):

    _template = Template("""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = L.control.locate(
                {{this.options | tojson}}
            ).addTo({{this._parent.get_name()}});
        {% endmacro %}
        """)

    def __init__(self, **kwargs):
        super(LocateControl, self).__init__()
        self._name = 'LocateControl'
        self.options = parse_options(**kwargs)

    def render(self, **kwargs):
        super(LocateControl, self).render(**kwargs)
        figure = self.get_root()
        assert isinstance(figure, Figure), ('You cannot render this Element '
                                            'if it is not in a Figure.')

        figure.header.add_child(
            CssLink(
                "https://cdnjs.cloudflare.com/ajax/libs/leaflet-locatecontrol/0.66.2/L.Control.Locate.min.css"))  # noqa
        figure.header.add_child(JavascriptLink(
            "https://cdnjs.cloudflare.com/ajax/libs/leaflet-locatecontrol/0.66.2/L.Control.Locate.min.js"))  # noqa

# Define coordinates of where we want to center our map.
boulder_coords = [51.042733, -86.900280]

# Using polygon markers with colors instead of default markers.
polygon_map = Map(location=boulder_coords, zoom_start=5, min_zoom=5)

#provides the popup labels for each GeoJSON object.
wmu_list = ['3','67','87D','68B','19','91A','89B','9B','21B','8','37','80','87C','69A-2','83B','73','69B',
                     '16B','42','23','30','83C','5','82B','32','76C','74B','86B','18B','58','59','69A-3','17','72A',
                     '75','25','85B','72B','87A','63A','55B','86A','68A','2','12A','82A','89A','64A','24','74A','7B',
                     '14','39','60','1C','7A','84','27','79C','90A','63B','35','50','6','11C','9A','78A','83A','47',
                     '81A','87E','92C','12B','93A','78B','20','26','70','77C','71','78E','88','90B','34','28','15A',
                     '53B','21A','13','49','87B','69A-1','78D','95','61','54','44','81B','91B','1B','94A','10','92D',
                     '46','77B','79D','38','41','85A','82C','33','43B','85C','51','92B','78C','1D','16A','94B','55A',
                     '1A','31','53A','64B','76E','76D','16C','18A','11A','92A','48','76B','57','4','43A','62','36',
                     '93C','56','22','76A','66B','65','45','77A','15B','11B','40','29','66A','93B']

df_rules = pd.read_csv('Dates.csv')

#['WMU', 'Animal', 'Category', 'Resident Dates', 'Non-resident Dates']
#iterates through the JSON object array and applies each individal object to the map with the required popup.
with open('Wildlife_Management_Unit.json') as json_file:
    data = json.load(json_file)
    for i in range(0, 151): #151
        geo = GeoJson(data=data["features"][i], style_function=lambda x: {'weight':1,'color':'#5BDAE9','dashArray':'1 3'},
                highlight_function=lambda x: {'weight':2,'color':'orange', 'dashArray':'1 7'},
                name='wmu 1').add_to(polygon_map)

        html = "<!DOCTYPE html><html><style> table,p { font-size: 13px; border-collapse: collapse; width: 100%; } </style><p><strong><center>Wildlife Management Unit "+str(wmu_list[i]) +\
               "</center></strong></p><br><table border='1'><tr><th>WMU</th><th>Animal</th><th>Category</th><th>Resident Dates</th>" \
               "<th>Non-Resident Dates</th></tr>"

        #list populated with lists of panda rows
        list_temp = []

        #get all rows that meet the wmu.
        for j in range(0, len(df_rules)):
            if df_rules.iloc[j, 0] == wmu_list[i]:
                list_temp.append(list(df_rules.iloc[j, :]))

        #update html table accordingly.
        for j in range (0, len(list_temp)):
            #creates a new html table row that is 5 cols wide.
            html += """
                    <tr>
                        <td>
                        """+str(list_temp[j][0])+"""
                        </td>
                        <td>
                        """+str(list_temp[j][1])+"""
                        </td>
                        <td>
                        """+str(list_temp[j][2])+"""
                        </td>
                        <td>
                        """+str(list_temp[j][3])+"""
                        </td>
                        <td>
                        """+str(list_temp[j][4])+"""
                        </td>
                    </tr>
                    """
        html+="</table></html>"
        iframe = IFrame(html=html, width=600, height=400)
        popup = Popup(iframe)
        popup.add_to(geo)

CLIENT_ID = 'H5BWH5B5EIER3USNJC5IDE5C32KD5YUVO3WDQ5IUOCCRARI5'
CLIENT_SECRET = 'PBIDS2HQ2ZRRPWAYU1JZNXCRTXR53SHFJH5VGJEHCWGHWKR0'
VERSION = '20191120'

#creates a client object
client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

#query keywords for POI's

#coordinates from varius regions around the province to effectively search the entire province.
cardinal_coords = ['43.358898,-80.839510', '49.465234,-81.336715','45.284421,-76.343714','48.945103,-90.040407',
                   '44.502762, -76.187975','46.601271, -80.827440','47.146339, -84.093960','47.881927, -84.654609',
                   '47.993280, -83.681636','49.353442, -87.034969','52.509904, -86.769346','53.964822, -90.582330',
                   '51.449819, -92.770166','49.105972, -92.436442', '54.667345, -84.233772']

#empty dataframe to hold POI data
df_poi = pd.DataFrame()

for coord in cardinal_coords:
    #conducts a FourSquare API call.
    json_dict = client.venues.search(params={'query': 'ServiceOntario', 'll': coord, 'radius' : '100000', 'state': 'ON', 'country': 'Canada', 'intent' : 'browse'})
    for i in range(0, len(json_dict['venues'])):
        venue = json_dict['venues'][i]
        print(venue)
        name = venue['name']
        lower = name.lower()
        if 'service' not in lower or 'ontario' not in lower:
            continue
        lat = venue['location']['lat']
        lng = venue['location']['lng']
        coord_formatted = str(lat) + "," + str(lng)
        geolocator = Nominatim(user_agent="hunting-map-capstone")
        address = geolocator.reverse(coord_formatted)
        data_list = [name,lat,lng,address]
        df_poi = df_poi.append(pd.DataFrame(columns=['Name','Latitude','Longitude', 'Address'], data=[data_list]))

print(df_poi.to_string())

feature_group = FeatureGroup(name='ServiceOntario Locations (Hunting Tags)')

for _, row in df_poi.iterrows():
    html = '<html><strong>Service Ontario</strong><br><p>'+str(row['Address']) + '</p><html>'
    iframe = IFrame(html=html, width=200, height=150)
    popup = Popup(iframe)
    Marker(location=[row['Latitude'], row['Longitude']],
           popup=popup).add_to(feature_group)

feature_group.add_to(polygon_map)

#adds the 'find my location' control.
LocateControl().add_to(polygon_map)

#display the map.
polygon_map.save(outfile= "wmumap.html")



FileNotFoundError: [Errno 2] No such file or directory: 'Wildlife_Management_Unit.json'

<h2>locate_control.py</h2>

In [ ]:
"""Add Locate control to folium Map.
Based on leaflet plugin: https://github.com/domoritz/leaflet-locatecontrol
"""

from branca.element import CssLink, Figure, JavascriptLink, MacroElement
from jinja2 import Template
from folium.utilities import parse_options

class LocateControl(MacroElement):
    """Control plugin to geolocate the user.
    This plugins adds a button to the map, and when it's clicked shows the current
    user device location.
    To work properly in production, the connection needs to be encrypted, otherwise browser will not
    allow users to share their location.
    Parameters
    ----------
    **kwargs
        For possible options, see https://github.com/domoritz/leaflet-locatecontrol
    Examples
    --------
    >>> m = folium.Map()
    # With default settings
    >>> LocateControl().add_to(m)
    # With some custom options
    >>> LocateControl(
    ...     position="bottomright",
    ...     strings={"title": "See you current location",
    ...              "popup": "Your position"}).add_to(m))
    For more info check:
    https://github.com/domoritz/leaflet-locatecontrol
    """

    _template = Template("""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = L.control.locate(
                {{this.options | tojson}}
            ).addTo({{this._parent.get_name()}});
        {% endmacro %}
        """)

    def __init__(self, **kwargs):
        super(LocateControl, self).__init__()
        self._name = 'LocateControl'
        self.options = parse_options(**kwargs)

    def render(self, **kwargs):
        super(LocateControl, self).render(**kwargs)
        figure = self.get_root()
        assert isinstance(figure, Figure), ('You cannot render this Element '
                                            'if it is not in a Figure.')

        figure.header.add_child(
            CssLink(
                "https://cdnjs.cloudflare.com/ajax/libs/leaflet-locatecontrol/0.66.2/L.Control.Locate.min.css"))  # noqa
        figure.header.add_child(JavascriptLink(
            "https://cdnjs.cloudflare.com/ajax/libs/leaflet-locatecontrol/0.66.2/L.Control.Locate.min.js"))  # noqa

<h2>html output: (static hosted with netlify) </h2>

<a href="https://modest-hawking-4ecc01.netlify.com/"> map.html </a>

<h2>Screenshot</h2>

<img src="https://imgur.com/vLEKkVl">
